In [ ]:
Author- Jay Shah
Date- 05/24

In [9]:
from torchvision import datasets
from torchvision.transforms import ToTensor

In [14]:
train_data=datasets.MNIST(
    root='data',
    train = True,
    transform=ToTensor(),
    download= True
)
test_data=datasets.MNIST(
    root='data',
    train = False,
    transform=ToTensor(),
    download= True
)


In [15]:
train_data.data.shape

torch.Size([60000, 28, 28])

In [16]:
test_data.data.shape


torch.Size([10000, 28, 28])

In [17]:
from torch.utils.data import DataLoader

In [19]:
loaders={
    'train': DataLoader(train_data,
                       batch_size=100,
                       shuffle=True,
                       num_workers=1),
    'test': DataLoader(test_data,
                       batch_size=100,
                       shuffle=True,
                       num_workers=1),
}

In [25]:
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import torch

In [33]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5) 
        self.conv2_drop = nn.Dropout2d() 
        self.fc1 = nn.Linear(320, 50)  
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))  
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))  
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [26]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CNN().to(device)

optimizer= optim.Adam(model.parameters(), lr=0.01)
loss_fn=nn.CrossEntropyLoss()

def train(epoch):
    model.train()
    total_correct = 0
    total_samples = 0
    for batch_idx, (data, target) in enumerate(loaders['train']):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        
        _, predicted = torch.max(output, 1)
        total_correct += (predicted == target).sum().item()
        total_samples += target.size(0)
        if batch_idx % 25 == 0:
            accuracy = 100.0 * total_correct / total_samples
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(loaders["train"].dataset)} ({100. * batch_idx / len(loaders["train"]):.0f}%)] Loss: {loss.item():.6f} Accuracy: {accuracy:.2f}%')

In [34]:
def test():
    model.eval()
    
    test_loss=0
    correct=0

    with torch.no_grad():
        for data, target in loaders['test']:
            data,target=data.to(device),target.to(device)
            output=model(data)
            test_loss=loss_fn(output,target).item()
            pred=output.argmax(dim=1,keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss/=len(loaders['test'].dataset)
    test_loss /= len(loaders['test'].dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(loaders["test"].dataset)} ({100. * correct / len(loaders["test"].dataset):.2f}%)')

In [35]:
for epoch in range(1,11):
    train(epoch)
    test()
    

Train Epoch: 1 [0/60000 (0%)] Loss: 0.352405 Accuracy: 89.00%
Train Epoch: 1 [2500/60000 (4%)] Loss: 0.360922 Accuracy: 91.46%
Train Epoch: 1 [5000/60000 (8%)] Loss: 0.291960 Accuracy: 91.18%
Train Epoch: 1 [7500/60000 (12%)] Loss: 0.204410 Accuracy: 90.97%
Train Epoch: 1 [10000/60000 (17%)] Loss: 0.293696 Accuracy: 90.95%
Train Epoch: 1 [12500/60000 (21%)] Loss: 0.332218 Accuracy: 90.88%
Train Epoch: 1 [15000/60000 (25%)] Loss: 0.368676 Accuracy: 90.81%
Train Epoch: 1 [17500/60000 (29%)] Loss: 0.397181 Accuracy: 90.80%
Train Epoch: 1 [20000/60000 (33%)] Loss: 0.554760 Accuracy: 90.97%
Train Epoch: 1 [22500/60000 (38%)] Loss: 0.124511 Accuracy: 90.96%
Train Epoch: 1 [25000/60000 (42%)] Loss: 0.248384 Accuracy: 90.87%
Train Epoch: 1 [27500/60000 (46%)] Loss: 0.279216 Accuracy: 90.92%
Train Epoch: 1 [30000/60000 (50%)] Loss: 0.321143 Accuracy: 90.90%
Train Epoch: 1 [32500/60000 (54%)] Loss: 0.294341 Accuracy: 90.97%
Train Epoch: 1 [35000/60000 (58%)] Loss: 0.260346 Accuracy: 91.02%
Train